# Introduction

- Name: Audrey Wanto
- Batch: BSD 002
- Dataset: [California Independent Medical Review Dataset](https://www.kaggle.com/datasets/prasad22/ca-independent-medical-review)
- Objective: Create a model inference based on the model created in the main notebook, and classify the new data.

# Model Inference

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

## Mount to Google Drive

In [2]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Unzip model and load model

In [3]:
# Unzip Model
!unzip -q '/content/drive/MyDrive/model.zip' -d model

replace model/fingerprint.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace model/keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace model/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace model/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace model/variables/variables.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [4]:
# Create path for model
model_path = '/content/model'

In [5]:
# Load model
model = tf.keras.models.load_model(model_path)

## Load data inference

In [6]:
# Read file
data = pd.read_csv('drive/My Drive/Data_inf.csv')

In [7]:
# Create dataframe
data = pd.DataFrame(data)
data

,Unnamed: 0,Reference ID,Report Year,Diagnosis Category,Diagnosis Sub Category,Treatment Category,Treatment Sub Category,Determination,Type,Age Range,Patient Gender,Findings
0,0,EI08-8281,2008,Central Nervous System/ Neuromuscular,Other,"Diagnostic Imaging, Screening and Testing",Other,Overturned Decision of Health Plan,Experimental/Investigational,51-64,Female,A 54-year-old female enrollee has requested fo...


In [8]:
# Create new dataframe with only 'Findings' columns
data_inf = data[['Findings']].copy()
data_inf

,Findings
0,A 54-year-old female enrollee has requested fo...


In [9]:
# Check summary of Model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 628)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 628, 128)          2659840   
                                                                 
 bidirectional (Bidirection  (None, 628, 64)           41216     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 628, 64)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 32)                10368     
 onal)                                                           
                                                        

In [10]:
# Predict the data inference
result = model.predict(data_inf)
result = np.where(result >= 0.5, 1, 0)

1/1 [==============================] - 2s 2s/step


From the main notebook, the results have been encoded into the following:

| Type | Target Value | Encoding |
| ---- | ------------ | -------- |
| Medical Necessity | 0 | [1, 0, 0] |
| Experimental/Investigational | 1 | [0, 1, 0] |
| Urgent Care | 2 | [0, 0, 1] |

And thus, it can be concluded that the result is of the following code cell

In [11]:
# Change result into list from array
result = result.tolist()

In [12]:
# Display Result
if result == [[1, 0, 0]]:
  print('Medical Necessity')
elif result == [[0, 1, 0]]:
  print('Experimental/Investigational')
elif result == [[0, 0, 1]]:
  print('Urgent Care')

Experimental/Investigational


From the original data_inf, it can be seen that the prediction matches with the actual `Type`, and thus the model works properly.